In [26]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

## Define constants to identify assets

In [27]:
# Futures contracts: Update when contracts roll
ES_CONTRACT = 'ESH24.CME'
US_CONTRACT = 'ZBH24.CBT'
CL_CONTRACT = 'CLF24.NYM'
NG_CONTRACT = 'NGF24.NYM'
GC_CONTRACT = 'GCG24.CMX'
SI_CONTRACT = 'SIH24.CMX'
EC_CONTRACT = '6EZ23.CME'
BP_CONTRACT = '6BZ23.CME'
W_CONTRACT = 'ZWH24.CBT'
C_CONTRACT = 'ZCH24.CBT'
S_CONTRACT = 'ZSF24.CBT'
BTC_CONTRACT = 'BTCZ23.CME'

# Stock tickers: Should seldom or never change
NVDA_TICKER = 'NVDA'

# Column labels for the dataframe
ES_HEADER = 'ESH24'
US_HEADER = 'USH24'
CL_HEADER = 'CLF24'
NG_HEADER = 'NGF24'
GC_HEADER = 'GCG24'
SI_HEADER = 'SIH24'
EC_HEADER = 'ECZ23'
BP_HEADER = 'BPZ23'
W_HEADER = 'WH24'
C_HEADER = 'CH24'
S_HEADER = 'SF24'
BTC_HEADER = 'BTCZ23'
NVDA_HEADER = 'NVDA'

## Function to retrieve a price series for one asset

In [28]:
def get_price_series(ticker_symbol, label):
    # Get today's and tomorrow's dates
    todays_date = datetime.today().strftime('%Y-%m-%d')
    tomorrows_date = (datetime.today() + timedelta(1)).strftime('%Y-%m-%d')
    # Get prices at 30-minute intervals
    ticker = yf.Ticker(ticker_symbol)
    time_df = ticker.history(start=todays_date, end=tomorrows_date, interval="30m")
    time_series = time_df['Close']
    # Rename the series to indicate the asset
    time_series.rename(label, inplace=True)
    return time_series

## Function to retrieve the open price of the 8:30 Central time bar (stock and agricultural markets open)

In [29]:
def get_open(ticker_symbol):
    # Get today's and tomorrow's dates
    todays_date = datetime.today().strftime('%Y-%m-%d')
    tomorrows_date = (datetime.today() + timedelta(1)).strftime('%Y-%m-%d')
    # Get prices at 30-minute intervals
    ticker = yf.Ticker(ticker_symbol)
    time_df = ticker.history(start=todays_date, end=tomorrows_date, interval="30m")
    # Get the 8:30 Central time open price
    stock_market_open = time_df.at_time("09:30")['Open']
    return stock_market_open

## Get prices for each asset

In [30]:
# Update ticker symbols when contracts roll
es_series = get_price_series(ES_CONTRACT, ES_HEADER)
us_series = get_price_series(US_CONTRACT, US_HEADER)
cl_series = get_price_series(CL_CONTRACT, CL_HEADER)
ng_series = get_price_series(NG_CONTRACT, NG_HEADER)
gc_series = get_price_series(GC_CONTRACT, GC_HEADER)
si_series = get_price_series(SI_CONTRACT, SI_HEADER)
ec_series = get_price_series(EC_CONTRACT, EC_HEADER)
bp_series = get_price_series(BP_CONTRACT, BP_HEADER)
w_series = get_price_series(W_CONTRACT, W_HEADER)
c_series = get_price_series(C_CONTRACT, C_HEADER)
s_series = get_price_series(S_CONTRACT, S_HEADER)
btc_series = get_price_series(BTC_CONTRACT, BTC_HEADER)
nvda_series = get_price_series(NVDA_TICKER, NVDA_HEADER)

In [31]:
# Concatenate all the series into a dataframe
prices = pd.concat([es_series, us_series, cl_series,
                   ng_series, gc_series, si_series,
                    ec_series, bp_series, w_series,
                    c_series, s_series, btc_series,
                    nvda_series], axis="columns")

### For stocks and agricultural markets that open at 8:30 Central time, get the open price of the 8:30 bar and use it as the closing price of the 8:00 bar.

In [32]:
if len(prices) >= 19:
    prices.iloc[18][ES_HEADER] = get_open(ES_CONTRACT)
    prices.iloc[18][W_HEADER] = get_open(W_CONTRACT)
    prices.iloc[18][C_HEADER] = get_open(C_CONTRACT)
    prices.iloc[18][S_HEADER] = get_open(S_CONTRACT)
    prices.iloc[18][NVDA_HEADER] = get_open(NVDA_TICKER)

C:\Users\steve\AppData\Local\Temp\ipykernel_12436\1135862775.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prices.iloc[18][ES_HEADER] = get_open(ES_CONTRACT)
C:\Users\steve\AppData\Local\Temp\ipykernel_12436\1135862775.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prices.iloc[18][W_HEADER] = get_open(W_CONTRACT)
C:\Users\steve\AppData\Local\Temp\ipykernel_12436\1135862775.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prices.iloc[18][C_HEADER] = get_open(C_CONTRACT)
C:\Users\steve\AppData\Local\Temp\ipykernel_12436\1135862775.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pri

### Convert to Central time and shift times ahead to the ends of the bars 

In [33]:
prices.tz_convert('US/Central').shift(30, freq='T')

,ESZ23,USH24,CLF24,NGF24,GCG24,SIH24,ECZ23,BPZ23,WH24,CH24,SF24,BTCZ23,NVDA
Datetime,,,,,,,,,,,,,
2023-12-06 23:30:00-06:00,4553.00,119.75000,69.769997,2.547,2042.900024,24.120001,1.07625,1.2548,632.75,484.00,1299.00,44550.0,NaN
2023-12-07 00:00:00-06:00,4552.00,119.84375,69.599998,2.548,2043.500000,24.135000,1.07655,1.2550,633.50,484.00,1302.50,44495.0,NaN
2023-12-07 00:30:00-06:00,4553.75,119.96875,69.750000,2.542,2045.300049,24.184999,1.07675,1.2555,633.25,484.25,1302.50,44450.0,NaN
2023-12-07 01:00:00-06:00,4551.00,119.68750,69.779999,2.523,2046.300049,24.205000,1.07675,1.2558,634.25,484.25,1302.25,44330.0,NaN
2023-12-07 01:30:00-06:00,4554.25,119.81250,69.870003,2.524,2045.800049,24.209999,1.07770,1.2566,636.25,485.00,1302.00,44420.0,NaN
2023-12-07 02:00:00-06:00,4554.50,119.78125,69.970001,2.520,2046.099976,24.170000,1.07845,1.2580,637.00,484.75,1303.50,44350.0,NaN
2023-12-07 02:30:00-06:00,4554.75,119.93750,69.730003,2.518,2049.199951,24.270000,1.07845,1.2582,638.00,485.50,1303.00,44125.0,NaN
2023-12-07 03:00:00-06:00,4550.75,119.84375,69.989998,2.518,2049.800049,24.299999,1.07840,1.2570,638.75,485.25,1299.75,44090.0,NaN
2023-12-07 03:30:00-06:00,4553.50,120.03125,69.870003,2.512,2048.100098,24.290001,1.07780,1.2568,640.00,485.00,1300.50,43510.0,NaN


## Get full O-H-L-C data for the S&P 500 contract

In [24]:
es_contract = yf.Ticker(ES_CONTRACT)
todays_date = datetime.today().strftime('%Y-%m-%d')
tomorrows_date = (datetime.today() + timedelta(1)).strftime('%Y-%m-%d')
es_ohlc = es_contract.history(start=todays_date, end=tomorrows_date, interval="30m")

In [25]:
es_ohlc.tz_convert('US/Central')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-12-06 23:00:00-06:00,4554.00,4554.25,4552.75,4553.00,2885,0.0,0.0
2023-12-06 23:30:00-06:00,4553.00,4553.25,4551.00,4552.00,4003,0.0,0.0
2023-12-07 00:00:00-06:00,4552.25,4554.75,4552.00,4553.75,3456,0.0,0.0
2023-12-07 00:30:00-06:00,4553.75,4554.00,4550.50,4551.00,2733,0.0,0.0
2023-12-07 01:00:00-06:00,4551.25,4554.75,4550.25,4554.25,6343,0.0,0.0
2023-12-07 01:30:00-06:00,4554.25,4555.50,4553.25,4554.50,4946,0.0,0.0
2023-12-07 02:00:00-06:00,4554.75,4557.25,4553.50,4554.75,14187,0.0,0.0
2023-12-07 02:30:00-06:00,4554.50,4555.00,4550.75,4550.75,11438,0.0,0.0
2023-12-07 03:00:00-06:00,4550.75,4554.00,4548.75,4553.50,15498,0.0,0.0
